# projet_location_neo4j

# generer mon localisation

In [3]:
import osmnx as ox
import folium
import geocoder
# Obtenir la localisation actuelle
location = geocoder.ip('me')
# Afficher les coordonnées géographiques
print(location.latlng)


[34.0331, -5.0003]


# afiicher mon localisation sout format html

In [4]:
import folium
from folium import plugins
from geopy.geocoders import Nominatim

# Coordonnées géographiques de Fès
fes_location = [34.0181, -5.0078]
# Création de la carte centrée sur Fès
fes_map = folium.Map(location=fes_location, zoom_start=12)
folium.Marker(
        location=location.latlng,
        popup="Votre position actuelle",
        icon=folium.Icon(color="red", icon="info-sign")
    ).add_to(fes_map)

# Afficher la carte
fes_map.save("fes_map.html")


# stocker mon localisation dans neo4j database local

In [5]:
from neo4j import GraphDatabase
# Informations de connexion à Neo4j
uri = "bolt://localhost:7687"
username = "neo4j"
password = "halima.123"
# Fonction pour créer un nœud de test
def create_test_node(tx):
    tx.run( ''' 
MATCH (location:Actuel_location {latitude:34.0331,longitude:-5.0003})
SET location.name="halima"
RETURN location
''')
# Connexion à la base de données Neo4j
driver = GraphDatabase.driver(uri, auth=(username, password))
# Création d'un nœud de test
with driver.session() as session:
    session.write_transaction(create_test_node)


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_21444\3194043741.py:17: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_test_node)


# generer un amlogithme d'AI pour calculer un lieu le plus proche

In [6]:
#Recherche de la distance euclidienne :
#Recherche du plus proche voisin (nearest neighbor) 
#Algorithme de Dijkstra 
#Algorithme A (A-star)*


# on vas utiliser A*

# extraire les hotels 

In [1]:
import osmnx as ox
# Coordonnées géographiques de Fès
fes_location = (34.0331, -5.0003)
# Télécharger les données OpenStreetMap pour la région de Fès
graph = ox.graph_from_point(fes_location, dist=10000, network_type='all')
# Extraire les nœuds correspondant aux hôtels
hotels = ox.geometries.geometries_from_place('Fès', tags={'tourism': 'hotel'})


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_16392\1700126786.py:7: UserWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in a future release.
  hotels = ox.geometries.geometries_from_place('Fès', tags={'tourism': 'hotel'})


In [2]:
hotels['name']

element_type  osmid     
node          262472277               Palais Jamai
              1139007346              Hôtel Mounia
              1313830498           Hôtel Volubilis
              1687718667               Hotel Royal
              1700815076             Hotel Cascade
                                     ...          
way           186942865     Hotel Perla فندق بيرلا
              376446937            Villa Agapanthe
              533783967                        NaN
              574668085              Les Merinides
              1225769807              Palais Amani
Name: name, Length: 99, dtype: object

# aficher les nome des hotels avec son coordonier geographiques (latitude,longitude) 

In [10]:
import pandas as pd
# Créer une liste vide pour stocker les données
data = []
# Parcourir les lignes du dataframe hotels
for index, row in hotels.iterrows():
    #  Extraire les coordonnées du point central du polygone
    centroid = row['geometry'].centroid
    latitude = centroid.y
    longitude = centroid.x
    # Ajouter les données de chaque ligne à la liste
    data.append({'name': row['name'], 'latitude': latitude, 'longitude': longitude})
# Créer le DataFrame à partir de la liste de données
db = pd.DataFrame(data)
db=db.dropna()
# Afficher le DataFrame
db

,name,latitude,longitude
0,Palais Jamai,34.069217,-4.974194
1,Hôtel Mounia,34.036309,-4.996784
2,Hôtel Volubilis,34.035278,-5.003424
3,Hotel Royal,34.043073,-5.001128
4,Hotel Cascade,34.061797,-4.983736
...,...,...,...
93,Fes Marriott Hotel Jnan Palace فندق ماريوت,34.031789,-5.002804
94,Hotel Perla فندق بيرلا,34.043789,-5.005235
95,Villa Agapanthe,34.001826,-4.951168
97,Les Merinides,34.069562,-4.981777


In [11]:
from geopy.distance import geodesic
import pandas as pd

# Coordonnées du point de référence
location = [34.0332, -5.0007]
# Calculer les distances entre le point de référence et chaque hôtel
db['distance'] = db.apply(lambda row: geodesic(location, (row['latitude'], row['longitude'])).kilometers, axis=1)
# Trier les hôtels par distance
db_sorted = db.sort_values(by='distance')
# Afficher les informations de l'hôtel le plus proche
hotel_le_plus_proche = db_sorted.iloc[0]
print("Informations sur l'hôtel le plus proche :")
print(hotel_le_plus_proche)
# Afficher les distances dans un DataFrame
distances_df = db_sorted[['name', 'distance']]
print("\nDistances par rapport au point de référence :")
distances_df


Informations sur l'hôtel le plus proche :
name         Fes Marriott Hotel Jnan Palace فندق ماريوت
latitude                                      34.031789
longitude                                     -5.002804
distance                                        0.24951
Name: 93, dtype: object

Distances par rapport au point de référence :


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_21444\2381355047.py:5: ResourceWarning: unclosed <socket.socket fd=4000, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('192.168.3.5', 29369), raddr=('34.117.186.192', 80)>
  location = [34.0332, -5.0007]


,name,distance
93,Fes Marriott Hotel Jnan Palace فندق ماريوت,0.249510
2,Hôtel Volubilis,0.341239
38,Maghreb Hôtel,0.482525
1,Hôtel Mounia,0.499668
34,Hotel central,0.500059
...,...,...
9,Hôtel Tijani,4.637451
42,tombouktoo,4.646501
0,Palais Jamai,4.685057
75,باب المدينة,4.867601


# stocker les hotels dans neo4j database

In [111]:
data

[{'name': 'Palais Jamai', 'latitude': 34.0692166, 'longitude': -4.9741942},
 {'name': 'Hôtel Mounia', 'latitude': 34.0363086, 'longitude': -4.9967841},
 {'name': 'Hôtel Volubilis', 'latitude': 34.0352784, 'longitude': -5.0034243},
 {'name': 'Hotel Royal', 'latitude': 34.0430735, 'longitude': -5.0011278},
 {'name': 'Hotel Cascade', 'latitude': 34.0617972, 'longitude': -4.9837364},
 {'name': 'Menzeh Zalagh', 'latitude': 34.0417077, 'longitude': -4.9946752},
 {'name': 'Hotel Zalagh Parc Palace',
  'latitude': 34.052244,
  'longitude': -5.0323491},
 {'name': 'Campini', 'latitude': 34.0580549, 'longitude': -4.9836516},
 {'name': 'Hotel sofia', 'latitude': 34.0404036, 'longitude': -5.0019849},
 {'name': 'Hôtel Tijani', 'latitude': 34.0676474, 'longitude': -4.9722382},
 {'name': 'Hotel Bab Boujloud',
  'latitude': 34.0612879,
  'longitude': -4.9837741},
 {'name': 'Palais Faraj', 'latitude': 34.0574076, 'longitude': -4.9761393},
 {'name': 'Hôtel Batha', 'latitude': 34.0601753, 'longitude': -4.

In [113]:
from neo4j import GraphDatabase
# Fonction pour créer un nœud pour chaque hôtel
def create_hotel_node(tx, hotel):
    query = (
        "CREATE (h:Hotel {name: $name, latitude: $latitude, longitude: $longitude})")
    tx.run(query, name=hotel['name'], latitude=hotel['latitude'], longitude=hotel['longitude'])
# Données des hôtels
hotels = data
# Connexion à la base de données Neo4j
uri = "bolt://localhost:7687"
username = "neo4j"
password = "halima.123"
driver = GraphDatabase.driver(uri, auth=(username, password))
# Création d'un nœud pour chaque hôtel
with driver.session() as session:
    for hotel in hotels:
        session.write_transaction(create_hotel_node, hotel)
# Fermeture de la connexion Neo4j
driver.close()


C:\Users\LENOVO\anaconda3\lib\site-packages\neo4j\_sync\driver.py:513: ResourceWarning: unclosed  BoltDriver: <neo4j._sync.driver.BoltDriver object at 0x0000022154A8E040>.
  _unclosed_resource_warn(self)
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_16732\1059459271.py:22: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_hotel_node, hotel)


# aficher les distance et les infos de l'hotel le plus proche

In [15]:
import networkx as nx
from sklearn.neighbors import BallTree
import pandas as pd

# Coordonnées du point de référence
location = [34.0332, -5.0007]

# Créer un graphe pondéré
G = nx.Graph()

# Ajouter le point de référence au graphe
G.add_node("reference", pos=(location[0], location[1]))

# Ajouter les hôtels comme des nœuds au graphe
for index, row in db.iterrows():
    G.add_node(row['name'], pos=(row['latitude'], row['longitude']))

# Calculer les distances entre le point de référence et chaque hôtel
for node1 in G.nodes():
    for node2 in G.nodes():
        if node1 != node2:
            pos1 = G.nodes[node1]['pos']
            pos2 = G.nodes[node2]['pos']
            distance = geodesic(pos1, pos2).kilometers
            G.add_edge(node1, node2, weight=distance)

# Appliquer l'algorithme A* pour trouver les distances les plus courtes de chaque hôtel au point de référence
shortest_paths = {}
for hotel in G.nodes():
    if hotel != "reference":
        shortest_paths[hotel] = nx.astar_path_length(G, "reference", hotel)

# Créer un DataFrame pour stocker les distances
distances_df = pd.DataFrame(shortest_paths.items(), columns=['Hotel', 'Distance'])

# Trier les hôtels par distance
distances_df = distances_df.sort_values(by='Distance')

# Afficher les informations de l'hôtel le plus proche
hotel_plus_proche = distances_df.iloc[0]
print("Informations sur l'hôtel le plus proche :")
print(db.loc[db['name'] == hotel_plus_proche['Hotel']])

# Afficher les distances dans un DataFrame
print("\nDistances par rapport au point de référence :")
distances_df


Informations sur l'hôtel le plus proche :
                                          name   latitude  longitude  distance
93  Fes Marriott Hotel Jnan Palace فندق ماريوت  34.031789  -5.002804   0.24951

Distances par rapport au point de référence :


,Hotel,Distance
78,Fes Marriott Hotel Jnan Palace فندق ماريوت,0.249510
2,Hôtel Volubilis,0.341239
35,Maghreb Hôtel,0.482525
1,Hôtel Mounia,0.499668
31,Hotel central,0.500059
...,...,...
9,Hôtel Tijani,4.637451
38,tombouktoo,4.646501
0,Palais Jamai,4.685057
67,باب المدينة,4.867601


# stocker les distanace dans neo4j

In [16]:
shortest_paths

{'Palais Jamai': 4.6850571673914505,
 'Hôtel Mounia': 0.49966807228483867,
 'Hôtel Volubilis': 0.3412385012468692,
 'Hotel Royal': 1.0959112268370592,
 'Hotel Cascade': 3.537724571589546,
 'Menzeh Zalagh': 1.2037654347183246,
 'Hotel Zalagh Parc Palace': 3.605956480252747,
 'Campini': 3.174740873025824,
 'Hotel sofia': 0.8078069974871641,
 'Hôtel Tijani': 4.6374512587144485,
 'Hotel Bab Boujloud': 3.4855995858898976,
 'Palais Faraj': 3.514721559553419,
 'Hôtel Batha': 3.4584905285196146,
 'Hotel Olympic': 0.7520149651625441,
 'Hotel de la Paix': 1.0707383778161839,
 'Grand Hôtel': 0.6871190550086005,
 'Nouzha': 0.6685021621468801,
 'Palais Medina Spa': 1.7551986157354513,
 'La Maison Verte': 4.121010035329349,
 'dar seffarine': 4.285159247943543,
 'Maison famille Tazi': 3.9336015331971024,
 'Hotel Wassim': 0.6639477619538582,
 'Hôtel Amour': 0.9048790376994081,
 'Hotel Nous Plazza': 0.516153417265548,
 'Hotel Savoy': 0.8741188418142993,
 'Hotel Zahrat Aljabal': 0.7126341113199129,
 'Ho

In [17]:
from neo4j import GraphDatabase

# Fonction pour créer un nœud pour chaque hôtel

query = (
       ''' WITH $shortest_paths AS shortestPaths
UNWIND keys(shortestPaths) AS hotelName
MATCH (h:Hotel {name: hotelName})
SET h.Distance = shortestPaths[hotelName]
'''
    )
# Connexion à la base de données Neo4j
uri = "bolt://localhost:7687"
username = "neo4j"
password = "halima.123"
driver = GraphDatabase.driver(uri, auth=(username, password))

with driver.session() as session:
    session.run(query, shortest_paths=shortest_paths)

# Fermeture de la connexion Neo4j
driver.close()


C:\Users\LENOVO\anaconda3\lib\site-packages\neo4j\_sync\driver.py:513: ResourceWarning: unclosed  BoltDriver: <neo4j._sync.driver.BoltDriver object at 0x0000025CF72700D0>.
  _unclosed_resource_warn(self)
C:\Users\LENOVO\anaconda3\lib\site-packages\neo4j\_sync\driver.py:518: DeprecationWarning: Relying on Driver's destructor to close the session is deprecated. Please make sure to close the session. Use it as a context (`with` statement) or make sure to call `.close()` explicitly. Future versions of the driver will not close drivers automatically.
  _deprecation_warn(


# ajouter relation entre local_location et l'hotel le plus proche

In [18]:
# Afficher les informations de l'hôtel le plus proche
hotel_plus_proche = distances_df.iloc[0]
print("Informations sur l'hôtel le plus proche :")
print(db.loc[db['name'] == hotel_plus_proche['Hotel']])

Informations sur l'hôtel le plus proche :
                                          name   latitude  longitude  distance
93  Fes Marriott Hotel Jnan Palace فندق ماريوت  34.031789  -5.002804   0.24951


In [22]:
from neo4j import GraphDatabase
# Fonction pour créer un nœud pour chaque hôtel
query = (
       '''MATCH (h:Hotel {name: "Fes Marriott Hotel Jnan Palace فندق ماريوت"}) ,(halima:Actuel_location)
 CREATE (h)-[:PLUS_PROCHE]->(halima)''' )
# Connexion à la base de données Neo4j
uri = "bolt://localhost:7687"
username = "neo4j"
password = "halima.123"
driver = GraphDatabase.driver(uri, auth=(username, password))
with driver.session() as session:
    session.run(query)
# Fermeture de la connexion Neo4j
driver.close()


# aficher le chemin dans le map

In [23]:
import requests
import folium
import polyline
# Points de départ et d'arrivée
start_point = (34.0188698, -4.9870592)
end_point = (34.013253, -4.982238)
# Clé API GraphHopper
api_key = "6675d14f-1060-46a5-9202-257fb62d348e"
# URL de l'API GraphHopper
url = f"https://graphhopper.com/api/1/route?point={start_point[0]},{start_point[1]}&point={end_point[0]},{end_point[1]}&vehicle=car&key={api_key}"
# Demander l'itinéraire à l'API GraphHopper
response = requests.get(url)
data = response.json()
# Obtenir les points encodés à partir de la réponse de l'API
encoded_points = data['paths'][0]['points']
# Décoder les points encodés pour obtenir les coordonnées du chemin
decoded_points = polyline.decode(encoded_points)
# Coordonnées géographiques de Fès
fes_location = [34.0181, -5.0078]
# Création de la carte centrée sur Fès
fes_map = folium.Map(location=fes_location, zoom_start=12)
folium.Marker(location=[34.013253 , -4.982238],popup="Hotel le plus proche",icon=folium.Icon(color="green", icon="info-sign")).add_to(fes_map)
folium.Marker(location=[34.0188698, -4.9870592],popup="Votre position actuelle",icon=folium.Icon(color="red", icon="info-sign")).add_to(fes_map)
# Tracer le chemin sur la carte
folium.PolyLine(locations=decoded_points,color='blue',weight=5,opacity=0.7).add_to(fes_map)
# Afficher la carte
fes_map.save("chemin_drive.html")

